# Описание ДЗ1.

На основе семинара 1 предложите 2 метода улучшения построения эмбеддингов вопросов на основе word vectors.

За задание можно получить максимум 10 баллов.

За каждый метод можно получить максимум 5 баллов.
Разбалловка:
*   Воспроизводимость и читабельность кода - 1 балл.
*   Корректность метода - 1 балл.
*   Описание метода в техническом отчете - 2 балла.
*   Иновационность - 1 балл.


# 1. Информация о сабмите

***Напишите здесь** ваше ФИО*

# 2. Технический отчет

***Введите сюда** подробное описание предложенных методов и экспериментов, с помощью которых вы пришли именно к выбору этих методов. НЕ вставляйте код в эту часть. Описание должно состоять минимум из 2-4 абзацев и содержать следующее: тип модели, параметры, как вы выбрали параметры, какие дальнейшие модификации готовых решений и т.д. вы использовали. Сюда можно включить, например, некоторые хитрости вашей предварительной обработки, описание моделей и мотивацию их использования, описание деталей процесса обучения. Если нужно, вставьте сюда графики, математические формулы.*

*Балл за «инновационность» будет присваиваться на основе содержания этой части. Если ваше отличие от бейзлайна это просто почистить тексты от стоп-слов или поменять одну базовую модель для построения word embeddings на другую, этот балл будет 0. Пробуйте разные подходы, модели, экспериментируйте с предварительной обработкой, параметрами и т. д. Можно переопределить уже существующий подход. Это нормально, что некоторые из ваших экспериментов не сработали так, как вы ожидали. Покажите нам, что вы проявили творческий подход и провели несколько экспериментов.*

# 3. *Code*

*Введите сюда весь код, использованный для получения результатов. Добавьте несколько комментариев и подразделов для навигации по вашему решению.*

*В этой части вам предстоит самостоятельно разработать решение задачи и предоставить воспроизводимый код:*
- *Использование Python 3;*
- *Содержит код для установки всех зависимостей;*
- *Содержит код для загрузки всех используемых наборов данных*;
- *Содержит код для воспроизведения ваших результатов (другими словами, если проверяющий загрузит ваш блокнот, он сможет выполнить код по ячейкам и получить результаты эксперимента, как описано в разделе методологии)*.


*В результате ваш код будет оценен по следующим критериям:*
- ***Читаемость**: ваш код должен быть хорошо структурирован, желательно с указанием частей вашего подхода (предварительная обработка, обучение модели, тестирование модели и т. д.).*
- ***Воспроизводимость**: ваш код должен воспроизводиться без ошибок в режиме «Выполнить все» (получение экспериментальной части).*


## 3.1 Импорт библиотек

In [1]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import gensim.downloader as api
# and some other your dependencies

## 3.2 Загрузка данных

In [2]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

data = list(open("./quora.txt", encoding="utf-8"))

--2024-05-23 16:51:17--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2024-05-23 16:51:17--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5b87f9a7e0f0fca29bb7913e13.dl.dropboxusercontent.com/cd/0/get/CTcIXlZw5k-MRew7ADyDkVqijBcRzdVmQ0dvjPeFcPwyFF_hWK2wzEHWEU1F0EotoOJoYneSDjibu6e5SCskR_47M8O6fV5nB3CvrjmFSrHL_YJkOBujcnWpShpQ9LxmN4o/file?dl=1# [following]
--2024-05-23 16:51:17--  https://uc5b87f9a7e0f0fca29bb7913e13.dl.dropboxusercontent.com/cd/0/get/CTcIXlZw5k-MRew7ADyDkVqijBcRzdVmQ0dvjPeFcPwyFF_hWK2wzEHWEU1F0EotoOJoYneSDjibu6e5SCskR_47M8O6fV5nB3CvrjmFSrHL_YJkOBujcnWpShpQ9LxmN4

## 3.3. Модель

In [3]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    vector = np.zeros([model.vector_size], dtype='float32')
    word_vectors = []

    data_tok = tokenizer.tokenize(phrase.lower())
    word_vectors = np.array([model.get_vector(word) for word in data_tok if word in model.key_to_index.keys()])

    if word_vectors.size != 0:
        vector = np.average(word_vectors, axis=0)

    return vector

In [4]:
tokenizer = WordPunctTokenizer()
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [5]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

## 3.4. Применение модели

In [6]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    from scipy.spatial.distance import cosine
    from tqdm import tqdm
    emb = get_phrase_embedding(query)
    similarities = np.array([1 - cosine(emb, embedding) for embedding in tqdm(data_vectors)])
    indices = np.argpartition(-similarities, range(10))[:10]

    return [data[ind] for ind in indices]

In [7]:
find_nearest(query="How does Trump?", k=10)

  6%|▋         | 33750/537272 [00:03<00:21, 23234.67it/s]/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 537272/537272 [00:27<00:00, 19871.31it/s]


['AOSDHIADSOIHADSO DASODASHDASOH\n',
 'parisflatlist\n',
 'What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n']